In [1]:
import os
%pwd
os.chdir("../")
%pwd


'd:\\Data Science\\END to END Proj\\BloodGroupClassifierBasedonFingerPrint'

In [2]:
## ENTITY
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path

In [3]:
from src.BloodGroupClassifier.constant import *
from src.BloodGroupClassifier.utils.common import read_yaml,create_directories 

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [5]:
import os
import urllib.request as request
import zipfile
import pandas as pd
from BloodGroupClassifier import logger
from BloodGroupClassifier.utils.common import get_size

In [12]:
import os
import zipfile
from pathlib import Path
from urllib import request

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> bool:
        """Downloads the file from source_URL to local_data_file if not present.
        Returns True if downloaded, False if already exists."""
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
        
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.source_URL}...")
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"Download completed to: {filename}")
            logger.debug(f"Download headers: {headers}")
            return True
        else:
            logger.info(f"File already exists at {self.config.local_data_file}, size: {get_size(Path(self.config.local_data_file))}")
            return False

    def extract_zip_file(self):
        """Extracts the downloaded zip file to unzip_dir"""
        logger.info(f"Extracting zip file from {self.config.local_data_file} to {self.config.unzip_dir}")
        os.makedirs(self.config.unzip_dir, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)

        logger.info(f"Successfully extracted to {self.config.unzip_dir}")

    def run(self):
        """Skip entire ingestion if zip file already exists"""
        if not os.path.exists(self.config.local_data_file):
            downloaded = self.download_file()
            if downloaded:
                self.extract_zip_file()
        else:
            logger.info(f"File already exists at {self.config.local_data_file}, skipping download and extraction.")


In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    # Only download and extract if file doesn't exist
    if not os.path.exists(data_ingestion_config.local_data_file):
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
    else:
        logger.info(
            f"File already exists at {data_ingestion_config.local_data_file}. Skipping download and extraction."
        )
except Exception as e:
    raise e


[2025-07-29 21:35:05,155: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-29 21:35:05,156: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-29 21:35:05,156: INFO: common: created directory at: artifacts]
[2025-07-29 21:35:05,156: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-29 21:35:05,169: INFO: 74520977: File already exists at artifacts/data_ingestion/FingerPrint.zip. Skipping download and extraction.]
